In [1]:
import os
import warnings

os.environ["WANDB_MODE"] = "dryrun"
warnings.filterwarnings('ignore')

In [2]:
!pip install git+https://github.com/csebuetnlp/normalizer
!pip install jiwer

  Cloning https://github.com/csebuetnlp/normalizer to /tmp/pip-req-build-60ka3f73
  Running command git clone --filter=blob:none --quiet https://github.com/csebuetnlp/normalizer /tmp/pip-req-build-60ka3f73
  Resolved https://github.com/csebuetnlp/normalizer to commit d405944dde5ceeacb7c2fd3245ae2a9dea5f35c9
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 185.0/185.0 kB 3.4 MB/s eta 0:00:0000:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for normalizer: filename=normalizer-0.0.1-py3-none-any.whl size=6859 sha256=f4dfcdcbbba9e5ae5d0b810cb50fcd38c5981d5ea23a919a558e83cfe6b2902b
  Stored in directory: /tmp/pip-ephem-wheel-cache-d8o8besc/wheels/2e/79/9c/cd96d490298305d51d2da11484bb2c25fd1f759a6906708282
  Created wheel for emoji: filename=emoji-1.4.2-py3-none-any.whl size=186460 sha256=8c0559125ca2d7a495cd94b412dd737cb82

In [3]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
import torch
import numpy as np
from torch.utils.data import Dataset
from normalizer import normalize

2024-04-25 03:08:53.604725: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-25 03:08:53.604831: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-25 03:08:53.736588: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
import pandas as pd

train_df = pd.read_csv("/kaggle/input/bnsecdataset/BNSECDataset/train.csv")
test_df = pd.read_csv("/kaggle/input/bnsecdataset/BNSECDataset/test.csv")

train_df.iloc[0].Input, train_df.iloc[0].Target

('রিয়াল ভাবটে পরে তারা আদলে 1111111111 পয়েন্টে এগিয়ে ।',
 'রিয়াল ভাবতে পারে তারা আসলে 1111111111 পয়েন্টে এগিয়ে ।')

In [5]:
print(train_df.isnull().sum())
print(train_df.duplicated().sum())

Target    0
Input     0
dtype: int64
1630


In [6]:
train_df.dropna(inplace=True)
test_df.dropna(inplace = True)

In [7]:
train_df.shape

(1356300, 2)

In [8]:
from tqdm import tqdm

sentences_with_errors_train = [
    normalize(erroneous_sentence) for erroneous_sentence in tqdm(train_df['Input'], desc="Processing training data")
]
corrected_sentences_train = [
    normalize(target_sentence) for target_sentence in tqdm(train_df['Target'], desc="Processing training targets")
]

sentences_with_errors_test = [
    normalize(erroneous_sentence) for erroneous_sentence in tqdm(test_df['Input'], desc="Processing test data")
]
corrected_sentences_test = [
    normalize(target_sentence) for target_sentence in tqdm(test_df['Target'], desc="Processing test targets")
]

# len(corrected_sentences_train)
# print(corrected_sentences_train)
# sentences_with_errors_train[0], corrected_sentences_train[0]

Processing test targets: 100%|██████████| 899/899 [00:00<00:00, 4279.41it/s]


In [9]:
# model_checkpoint = "google-t5/t5-base"
model_checkpoint = "csebuetnlp/banglat5"
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=False)

config.json:   0%|          | 0.00/659 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/990M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.83k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [11]:
text = "আমি আমার মন ঠিক করেছি। আমি একটা ট্যাটু আঁকছি।"
input_encoding = tokenizer(
    text=normalize(text), 
    return_tensors='pt', 
    max_length=512, 
    padding='max_length', 
    truncation=True
)

# Generate corrected sentence
generated_ids = model.generate(
    input_encoding["input_ids"]
)

# Decode the generated sequence into text
generated_text = tokenizer.decode(
    generated_ids[0], 
    skip_special_tokens=True
)
print("generated sentence:", generated_text)

generated sentence: আমি আমার মন ঠিক করেছি। আমি একটা ট্যাটু আঁকছি। আমি আমার মন ঠিক করেছি।


In [12]:
class GrammarCorrectionDataset(Dataset):
    def __init__(self, sentences_with_errors, corrected_sentences, tokenizer, split_type='train'):
        self.sentences_with_errors = sentences_with_errors
        self.corrected_sentences = corrected_sentences
        self.tokenizer = tokenizer
        self.split_type = split_type
        self.vocab_size = tokenizer.vocab_size
        self.max_sequence_length = 128  # Adjust as needed

    def __len__(self):
        return len(self.sentences_with_errors)

    def __getitem__(self, idx):
        sentence_with_error = self.sentences_with_errors[idx]
        corrected_sentence = self.corrected_sentences[idx]

        input_encoding = self.tokenizer(
            sentence_with_error, 
            return_tensors='pt', 
            max_length=self.max_sequence_length, 
            padding='max_length', 
            truncation=True
        )
        label_encoding = self.tokenizer(
            corrected_sentence, 
            return_tensors='pt', 
            max_length=self.max_sequence_length, 
            padding='max_length', 
            truncation=True
        )

        return {
            "input_ids": input_encoding["input_ids"].squeeze(), 
            "labels": label_encoding["input_ids"].squeeze()
        }

In [ ]:
batch_size = 8  # Adjust as needed
model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    output_dir=f"{model_name}-finetuned-grammar-correction",
    evaluation_strategy="epoch",
    learning_rate=1e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=1,
    num_train_epochs=1,  # Adjust as needed
    predict_with_generate=True,
    skip_memory_metrics=False,
    push_to_hub=False,
)

# prepare dataset
train_dataset = GrammarCorrectionDataset(
    sentences_with_errors_train,
    corrected_sentences_train,
    tokenizer,
    split_type='train'
)
test_dataset = GrammarCorrectionDataset(
    sentences_with_errors_test, 
    corrected_sentences_test, 
    tokenizer, 
    split_type='test'
)

data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer, 
    model=model
)

trainer = Seq2SeqTrainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
)

trainer.train()

Epoch,Training Loss,Validation Loss


In [ ]:
# # Assuming `model` and `tokenizer` are already defined and trained
# sentence_with_error = "আমি ভ্যাট খাই"  # Example sentence with error
# input_encoding = tokenizer(normalize(sentence_with_error), return_tensors='pt', max_length=128, padding='max_length', truncation=True)
# generated_ids = model.generate(input_encoding["input_ids"])
# corrected_sentence = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
# print("Corrected sentence:", corrected_sentence)

In [ ]:
model_output_dir = "./trained_grammar_correction_model"
model.save_pretrained(model_output_dir)
tokenizer.save_pretrained(model_output_dir)

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer


loaded_model = AutoModelForSeq2SeqLM.from_pretrained(model_output_dir)
loaded_tokenizer = AutoTokenizer.from_pretrained(model_output_dir, use_fast=False)

In [ ]:
# sentence_with_error = "রিয়াল ভাবটে পরে তারা আদলে 1111111111 পয়েন্টে এগিয়ে ।"
sentence_with_error = "আমি ভ্যাট খাই"
input_encoding = loaded_tokenizer(
    text=normalize(sentence_with_error), 
    return_tensors='pt', 
    max_length=128, 
    padding='max_length', 
    truncation=True
)

# Generate corrected sentence
generated_ids = loaded_model.generate(
    input_encoding["input_ids"]
)

# Decode the generated sequence into text
corrected_sentence = loaded_tokenizer.decode(
    generated_ids[0], 
    skip_special_tokens=True
)
print("Corrected sentence:", corrected_sentence)